In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import mplfinance as mpf
import matplotlib.pyplot as plt
import talib as ta


stock_codes1 = ["000001.SS","399001.SZ", "^HSI","^STI","^N225","^KS11","^IXIC","^GSPC","^DJI","IWM",\
                "^GSPTSE","^BVSP","^STOXX50E","^GDAXI", "^FTSE", "^FCHI", "^IBEX","^AXJO","^AORD","^NZ50"]

def calculate_ma(df, ma_periods=[5,10,20,30,60]):
    for period in ma_periods:
        df[f'{period}-day moving average'] = df['Close'].rolling(period).mean()
    return df

stock_codes = stock_codes1[-2:]

for each in stock_codes:
    data = yf.Ticker(each).history(start="2015-01-01", end="2019-12-31", interval="1d")
    df = data.loc[:, ['Open', 'High', 'Low', 'Close', 'Volume']]
    df = df.rename_axis('Date')
    if len(df) == 0:
        continue

    # MA
    df = calculate_ma(df)

    year = list(set(df.index.year))

    # Calculate MACD
    df['macd'], df['macd_signal'], df['macd_hist'] = ta.MACD(df['Close'].values, fastperiod=12, slowperiod=26, signalperiod=9)
    print(each, df, sep='\n')

    for i in year:
        temp_y = df[df.index.year == i]
        week = list(set(temp_y.index.week))

        for j in week:
            temp_w = temp_y[temp_y.index.week == j]
            has_nan = temp_w.isna().any().any().any()
            if has_nan == 1:
                continue

            # MA chart
            ma_periods = [5, 10, 20, 30, 60]
            ma_plots = [mpf.make_addplot(temp_w[f'{period}-day moving average'], panel=0, ylabel='', title='') for period in ma_periods]

            min_date = temp_w.index.min()
            min_date_tz = min_date.tzinfo
            start_date = pd.to_datetime("20150101").tz_localize(min_date_tz)

            if min_date > start_date:
                # MACD chart
                ap = []
                ap.append(mpf.make_addplot(temp_w['macd'], panel=2, color='fuchsia'))
                ap.append(mpf.make_addplot(temp_w['macd_signal'], panel=2, color='b'))
                colors = ['g' if v <= 0 else 'r' for v in temp_w["macd_hist"]]
                ap.append(mpf.make_addplot(temp_w['macd_hist'], panel=2, type='bar', color=colors))
                s = mpf.make_mpf_style(base_mpf_style='yahoo', mavcolors=['c', 'lime'])
                save_path = r'D:\硕士\project\Progress 3\Data by weeks/'
                filename = f"chart_{each}_{i}_{j}.png"
                mpf.plot(temp_w, addplot=ap+ma_plots, figratio=(40,20), panel_ratios=[2,2,2], type='candle', style=s, volume=True, volume_panel=1, savefig=save_path + filename,
                         ylabel='', ylabel_lower='', title='')

^AORD
                                  Open         High          Low        Close  \
Date                                                                            
2015-01-02 00:00:00+11:00  5388.600098  5415.000000  5366.399902  5415.000000   
2015-01-05 00:00:00+11:00  5415.000000  5447.600098  5409.399902  5429.500000   
2015-01-06 00:00:00+11:00  5429.500000  5429.500000  5321.500000  5346.200195   
2015-01-07 00:00:00+11:00  5346.200195  5348.899902  5301.200195  5334.500000   
2015-01-08 00:00:00+11:00  5334.500000  5373.399902  5334.500000  5359.399902   
...                                ...          ...          ...          ...   
2019-12-20 00:00:00+11:00  6942.600098  6950.700195  6918.200195  6924.399902   
2019-12-23 00:00:00+11:00  6924.399902  6924.399902  6894.700195  6894.700195   
2019-12-24 00:00:00+11:00  6894.799805  6906.399902  6886.799805  6906.399902   
2019-12-27 00:00:00+11:00  6906.399902  6936.500000  6906.399902  6936.299805   
2019-12-30 00:00:00+11

C:\Users\Tommy\AppData\Local\Temp\ipykernel_20320\2458953085.py:37: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  week = list(set(temp_y.index.week))
C:\Users\Tommy\AppData\Local\Temp\ipykernel_20320\2458953085.py:40: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  temp_w = temp_y[temp_y.index.week == j]
C:\Users\Tommy\AppData\Local\Temp\ipykernel_20320\2458953085.py:40: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear

^NZ50
                                   Open          High           Low  \
Date                                                                  
2015-01-05 00:00:00+13:00   5577.379883   5604.859863   5568.279785   
2015-01-06 00:00:00+13:00   5573.149902   5602.600098   5557.430176   
2015-01-07 00:00:00+13:00   5549.220215   5566.080078   5541.620117   
2015-01-08 00:00:00+13:00   5561.709961   5580.899902   5558.060059   
2015-01-09 00:00:00+13:00   5589.680176   5608.470215   5574.049805   
...                                 ...           ...           ...   
2019-12-20 00:00:00+13:00  11480.610352  11512.269531  11470.309570   
2019-12-23 00:00:00+13:00  11482.290039  11585.990234  11482.290039   
2019-12-24 00:00:00+13:00  11547.230469  11642.780273  11543.339844   
2019-12-27 00:00:00+13:00  11593.690430  11624.740234  11577.089844   
2019-12-30 00:00:00+13:00  11602.120117  11628.820312  11556.450195   

                                  Close    Volume  5-day moving averag

C:\Users\Tommy\AppData\Local\Temp\ipykernel_20320\2458953085.py:37: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  week = list(set(temp_y.index.week))
C:\Users\Tommy\AppData\Local\Temp\ipykernel_20320\2458953085.py:40: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  temp_w = temp_y[temp_y.index.week == j]
C:\Users\Tommy\AppData\Local\Temp\ipykernel_20320\2458953085.py:40: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear

In [3]:
import numpy as np
import os
from PIL import Image

image_dir = r"D:\硕士\project\Progress 3\Data by weeks"
output_dir = r"D:\硕士\project\Progress 3\good_data"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

file_list = os.listdir(image_dir)
image_list = [i for i in file_list if i.endswith('.png')]
image_list.sort()

for image_file in image_list:
    image_path = os.path.join(image_dir, image_file)
    output_path = os.path.join(output_dir, image_file)
    
    img = Image.open(image_path)
    cropped_img = img.crop((208, 71, 1037, 474))
    cropped_img.save(output_path)

print("Image cropping complete!")

Image cropping complete!


In [4]:
import pandas as pd
import os
import calendar
import yfinance as yf
from datetime import datetime, timedelta

image_dir = r"D:\硕士\project\Progress 3\good_data"
output_csv = r"D:\硕士\project\Progress 3\dataset\labels.csv"

# Create an empty DataFrame to store the labels
labels_df = pd.DataFrame(columns=["Stock Code", "Year", "Week"])

# Iterate through the images in the directory
image_list = os.listdir(image_dir)
for image_file in image_list:
    if image_file.endswith('.png'):
        # Extract stock code, year, and week from the image filename
        stock_code, year, week = image_file.split("_")[1:4]
        week = week.replace(".png", "")
        
        # Format the week string with leading zero if necessary
        week_str = week if len(week) == 2 else f"0{week}"
        
        # Calculate the start and end dates based on the year and week number
        start_date = datetime.strptime(f"{year}-W{week_str}-1", "%Y-W%W-%w")
        end_date = start_date + timedelta(days=6)
        
        # Load the stock price data from Yahoo Finance
        data = yf.Ticker(stock_code).history(start=start_date, end=end_date)
        if data.empty:
            continue  # Skip to the next image if there is no data
        
        df = data.loc[:, ['Open', 'High', 'Low', 'Close', 'Volume']]
        df = df.rename_axis('Date')
        stock_data = df
                
        # Calculate the label based on the price change
        first_price = stock_data.iloc[0]["Close"]
        last_price = stock_data.iloc[-1]["Close"]
        label1 = 1 if last_price > first_price else 0
        label2 = (last_price-first_price)/first_price
        
        # Append the label to the DataFrame
        new_row = pd.DataFrame([{"Stock Code": stock_code, "Year": year, "Week": week, "Rise": label1, "Return": label2}])
        labels_df = pd.concat([labels_df, new_row], ignore_index=True)

# Save the labels to a CSV file
labels_df.to_csv(output_csv, index=False)

print("Label generation complete!")

Label generation complete!


In [5]:
# unzip the file
import os
from zipfile import ZipFile

extracted_dir = "./"
with ZipFile(r"D:/硕士/project/Progress 3/good_data.zip", 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)
    

import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

image_dir = r"D:\硕士\project\Progress 3\good_data"
csv_file = "D:\硕士\project\Progress 3\dataset\labels.csv"

# Load the CSV data
data_df = pd.read_csv(csv_file)

# Check the number of missing values in each column
data_df.isnull().sum()

# Create empty lists to store the image data and labels
image_data = []
labels = []

# Iterate through the CSV rows
for index, row in data_df.iterrows():
    stock_code = row["Stock Code"]
    year = row["Year"]
    week = row["Week"]

    # Construct the image filename
    image_filename = f"chart_{stock_code}_{year}_{week}.png"
    image_path = os.path.join(image_dir, image_filename)

    # Open and convert the image to binary format
    image = Image.open(image_path)
    image = image.resize((207, 101))
    image = image.convert('L')
    image_array = np.array(image)

    # Append the image data and label to the lists
    image_data.append(image_array)
    labels.append(row["Rise"])

# Convert the lists to NumPy arrays
image_data = np.array(image_data)
labels = np.array(labels)

# Split the data into training and testing sets (80:20 ratio)
split_ratio = 0.8
split_index = int(len(image_data) * split_ratio)

image_train_val_data = image_data[:split_index]
label_train_val_data = labels[:split_index]
image_test_data = image_data[split_index:]
label_test_data = labels[split_index:]

# Print the shape of the training and testing sets
print("Training Images:", image_train_val_data.shape)
print("Training Labels:", label_train_val_data.shape)
print("Testing Images:", image_test_data.shape)
print("Testing Labels:", label_test_data.shape)

import copy
data_df_temp = copy.deepcopy(data_df)

Training Images: (396, 101, 207)
Training Labels: (396,)
Testing Images: (100, 101, 207)
Testing Labels: (100,)
